##Functions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
import os
os.chdir('/content/drive/MyDrive/PERSONAL AGJM/PROYECTOS ML Y ESTAD./A B TESTING/Data')

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statistics as st

def plot_dens(data, treatment_indicator, variables=None, xlabel='Value', title_prefix='Density by Treatment - '):
    """
    Plot density distributions of multiple variables based on treatment indicator.

    Parameters:
    - data: Pandas DataFrame, the dataset containing variables of interest.
    - treatment_indicator: 1D NumPy array or Pandas Series, treatment indicator (0 for control, 1 for treatment).
    - variables: List of strings, variables to plot. If None, all numeric columns in 'data' are used.
    - xlabel: String, label for the x-axis (default: 'Value').
    - title_prefix: String, prefix for the titles of the plots (default: 'Density Distributions by Treatment - ').

    Returns:
    - None (displays the plots).
    """

    # If variables are not specified, use all numeric columns
    if variables is None:
        variables = data.select_dtypes(include=np.number).columns

    num_variables = len(variables)
    num_cols = 2
    num_rows = int(np.ceil(num_variables / num_cols))

    # If there's only one variable, adjust the layout for a single plot
    if num_variables == 1:
        fig, ax = plt.subplots(figsize=(7, 4))
        axes = [ax]
    else:
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 8))
        axes = axes.flatten()

    # Loop through each variable and plot density distributions
    for i, variable in enumerate(variables):
        ax = axes[i]

        # Separate data based on treatment indicator
        control_group = data.loc[treatment_indicator == 0, variable]
        treatment_group = data.loc[treatment_indicator == 1, variable]
#grey,#orange
        # Plot density distributions using seaborn's kdeplot
        sns.kdeplot(control_group, color='grey', label='Control', ax=ax,fill=True,alpha=0.2)
        sns.kdeplot(treatment_group, color='red', label='Treatment', ax=ax,fill=True,alpha=0.1)
        #label=f'Control - {variable}'

        # Add labels and title
        ax.set_xlabel(xlabel,fontsize=9)
        ax.set_ylabel('Density',fontsize=9)
        #ax.set_title(title_prefix + variable,fontsize=13)
        ax.set_title(variable,fontsize=14,fontweight='bold')
        ax.legend(fontsize=9)

    # Remove empty subplots for odd number of variables
    for i in range(num_variables, len(axes)):
        fig.delaxes(axes[i])

    # Adjust layout for better spacing
    plt.tight_layout()

    # Show the plot
    plt.show()



def check_balance(df,feat,flag_group):
  if df[feat].dtype.name=='object':
    sdf=[]
    aux_exposed=pd.DataFrame(df[df[flag_group]==1][feat].value_counts(1)).reset_index()
    aux_control=pd.DataFrame(df[df[flag_group]==0][feat].value_counts(1)).reset_index()

    for i in aux_control['index'].unique().tolist():

      prop_c=aux_control[aux_control["index"]==i][feat].values[0]
      prop_e=aux_exposed[aux_exposed["index"]==i][feat].values[0]
      res=(abs(prop_e-prop_c))/(np.sqrt((prop_e*(1-prop_e)+prop_c*(1-prop_c))*0.5))
      sdf.append({i:res})

  else:
    # calcule the mean to both grupus:

    mean_feat_control=df[df[flag_group]==0][feat].mean()
    mean_feat_exposed=df[df[flag_group]==1][feat].mean()

    # calcule the sd to both grupus:
    st_feat_control=np.var(df[df[flag_group]==0][feat])
    st_feat_exposed=np.var(df[df[flag_group]==1][feat])

    # Calculated two elements of fracction:
    num=mean_feat_control-mean_feat_exposed
    den=np.sqrt((st_feat_control+st_feat_exposed)*0.5)
    sdf=num/den

  return sdf

import numpy as np
from sklearn.metrics import pairwise_distances
import pandas as pd

def caliper_matching_with_covariates(treated_covariates, control_covariates, treated_propensity, control_propensity, caliper=0.005):
    """
    Perform caliper matching based on propensity scores and return matched pairs with covariates.

    Parameters:
    - treated_covariates: numpy array, covariates for treated units
    - control_covariates: numpy array, covariates for control units
    - treated_propensity: numpy array, propensity scores for treated units
    - control_propensity: numpy array, propensity scores for control units
    - caliper: float, maximum allowable difference in propensity scores (default is 0.05)

    Returns:
    - matched_data: DataFrame, matched pairs with covariates
    """
    # Calculate pairwise distances between treated and control propensity scores
    distances = pairwise_distances(treated_propensity.reshape(-1, 1), control_propensity.reshape(-1, 1))

    # Perform caliper matching
    matched_pairs = []

    for i in range(len(treated_propensity)):
        min_distance = np.min(distances[i, :])
        min_distance_index = np.argmin(distances[i, :])

        if min_distance <= caliper:
            matched_pairs.append((i, min_distance_index))

    # Extract covariates for matched pairs
    matched_treated_covariates = treated_covariates[[pair[0] for pair in matched_pairs]]
    matched_control_covariates = control_covariates[[pair[1] for pair in matched_pairs]]

    # Create a DataFrame with matched covariates
    matched_data = pd.DataFrame(np.hstack([matched_treated_covariates, matched_control_covariates]),
                                columns=[f't_{i}' for i in range(matched_treated_covariates.shape[1])] +
                                        [f'c_{i}' for i in range(matched_control_covariates.shape[1])])

    # Add propensity scores and treatment indicator to the DataFrame
    matched_data['Propensity_Treated'] = treated_propensity[[pair[0] for pair in matched_pairs]]
    matched_data['Propensity_Control'] = control_propensity[[pair[1] for pair in matched_pairs]]
    matched_data['Treatment'] = 1  # Indicator for treated units
    matched_data['Control'] = 0    # Indicator for control units

    return matched_data




## Reading a DataSet

In [ ]:
columns = ["training",   # Treatment assignment indicator
           "age",        # Age of participant
           "education",  # Years of education
           "black",      # Indicate whether individual is black
           "hispanic",   # Indicate whether individual is hispanic
           "married",    # Indicate whether individual is married
           "no_degree",  # Indicate if individual has no high-school diploma
           "re74",       # Real earnings in 1974, prior to study participation
           "re75",       # Real earnings in 1975, prior to study participation
           "re78"]       # Real earnings in 1978, after study end

file_names = ["http://www.nber.org/~rdehejia/data/nswre74_treated.txt",
              "http://www.nber.org/~rdehejia/data/nswre74_control.txt",
              "http://www.nber.org/~rdehejia/data/psid_controls.txt",
              "http://www.nber.org/~rdehejia/data/psid2_controls.txt",
              "http://www.nber.org/~rdehejia/data/psid3_controls.txt",
              "http://www.nber.org/~rdehejia/data/cps_controls.txt",
              "http://www.nber.org/~rdehejia/data/cps2_controls.txt",
              "http://www.nber.org/~rdehejia/data/cps3_controls.txt"]
files = [pd.read_csv(file_name, delim_whitespace=True, header=None, names=columns) for file_name in file_names]
lalonde = pd.concat(files, ignore_index=True)

In [ ]:
print("DataFrame dimension:",lalonde.shape)
lalonde.sample(3)

In [ ]:
print("Distribution of training indicator:")
lalonde["training"].value_counts()

In [ ]:
map_names={"age":"Edad","education":"tiempo_educacion","re74":"Ingreso_1974","re75":"Ingreso_1975"}
lalonde.rename(columns=map_names,inplace=True)

In [ ]:
lalonde.sample(3)

## Check density between group

In [ ]:
confounders=['Edad','tiempo_educacion','black','hispanic','married',
            'no_degree','Ingreso_1974','Ingreso_1975']
confounders2=['Edad','tiempo_educacion','Ingreso_1974','Ingreso_1975']

In [ ]:
diffs=[]
for c in confounders:
  diffs.append({check_balance(lalonde,c,"training"):c})
diffs

In [ ]:
plot_dens(lalonde,lalonde["training"],confounders2)

In [ ]:
lalonde["training"].value_counts()

## Fit a LR to stimate propensity scores

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, roc_auc_score,precision_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Feature_4 is the outcome

X = lalonde[confounders]
treatment = lalonde["training"]

# Create a binary outcome variable 'y' (replace this with your actual outcome variable)
#y = np.random.choice([0, 1], size=1000)

# Split the data into training and testing sets
X_train, X_test, treatment_train, treatment_test = train_test_split(
    X, treatment, test_size=0.2, random_state=42,stratify=treatment)


# Apply SMOTE to the training data
smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_train_smote, treatment_train_smote = smote.fit_resample(X_train, treatment_train)


# Scale the data
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train_smote)
#X_test_scaled = scaler.transform(X_test)

# Define the classifiers with hyperparameter grids
classifiers = {
    'Logistic Regression': (LogisticRegression(), {"max_iter":[1000],
                                                   "class_weight":[None,"balanced"]})#,
    #'Decision Tree': (DecisionTreeClassifier(), {'max_depth': [None, 3,5,10]})#,
    #'Random Forest': (RandomForestClassifier(), {'n_estimators': [10,50,100], 'max_depth': [None, 5, 10]})
}

# Fit each classifier with hyperparameter tuning, SMOTE, and cross-validation, and evaluate performance
results = {}
for name, (classifier, param_grid) in classifiers.items():
    grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring=make_scorer(roc_auc_score))
    grid_search.fit(X_train_smote, treatment_train_smote)

    # Get the best model from grid search
    best_model = grid_search.best_estimator_

    # Cross-validate the model
    cv_scores = cross_val_score(best_model, X_train_smote, treatment_train_smote, cv=5, scoring=make_scorer(roc_auc_score))

    # Predict propensity scores on the testing data
    propensity_scores = best_model.predict_proba(X_test)[:, 1]

    # Evaluate the performance using the Area Under the Receiver Operating Characteristic Curve (AUC score)
    auc_score = roc_auc_score(treatment_test, propensity_scores)   # YA HICE UN RELAJO POR QUE TENGO QUE CAMBIAR TODO A PRECISION, CAMIBIEN EN GRID SEARCH PERO EN LO DEMÁS NO, REVISAR!!!!

    # Store results
    results[name] = {'best_model': best_model, 'cv_scores': cv_scores, 'roc_auc_score': auc_score}

# Identify the best-performing model
best_model_info = max(results.items(), key=lambda x: np.mean(x[1]['cv_scores']))
best_model_name, best_model_info = best_model_info

# Print the results
print("Propensity Score Estimation Results:")
for name, info in results.items():
    print(f"{name}: Best AUC Score = {info['roc_auc_score']:.4f} (Cross-validated AUC: {np.mean(info['cv_scores']):.4f}) using {info['best_model']}")

print(f"\nBest Model: {best_model_name} with AUC Score = {best_model_info['roc_auc_score']:.4f} (Cross-validated AUC: {np.mean(best_model_info['cv_scores']):.4f})")


## Propensity score stimation over all dataset

In [ ]:
# Add the estimation of propensity score (ps)
best_model_ps=best_model_info["best_model"]
lalonde["ps"]=best_model.predict_proba(lalonde[confounders])[:, 1]

## Checking Overlap

In [ ]:
# Comparing the distributions of ps for both groups
plot_dens(lalonde,lalonde["training"],variables=["ps"])

## Saving de model

In [ ]:
# Save the model
import pickle
with open('best_model_ps.pkl', 'wb') as model_file:
    pickle.dump(best_model_ps, model_file)

## Caliper Matching

In [ ]:
cov=confounders+["re78"]

# Dataframes with covariates separated by training indicator
treat_c=lalonde[lalonde["training"]==1][cov].values
control_c=lalonde[lalonde["training"]==0][cov].values

# Values of propensity score stimation
treat_ps=lalonde[lalonde["training"]==1]["ps"].values
control_ps=lalonde[lalonde["training"]==0]["ps"].values

# Calculating treshold for caliper matching algorithm
caliper=st.stdev(lalonde["ps"])*0.25

In [ ]:
# Running Caliper Matching algoritm
matched_data = caliper_matching_with_covariates(treat_c, control_c,treat_ps, control_ps, caliper=0.01)

## Cleaning Matching output

In [ ]:
# Variables names acording to matching
vars_t=[x for x in matched_data.columns if x.startswith("t")]
vars_c=[x for x in matched_data.columns if x.startswith("c")]

# Separed the matched data  by Treatment:Training and Control: No Training
matched_control=matched_data[vars_c+["Control","Propensity_Control"]]
matched_treated=matched_data[vars_t+["Treatment","Propensity_Treated"]]

# Cols names for the clean matched data
#cols=confounders+["Training","ps"]

# Dictionaries to map
dic_control=dict(zip(matched_control.columns.tolist(),cov+["Training","ps"]))
dic_treated=dict(zip(matched_treated.columns.tolist(),cov+["Training","ps"]))
# Rename columns
matched_control.rename(columns=dic_control,inplace=True)
matched_treated.rename(columns=dic_treated,inplace=True)

# Concat the matched information
matched_final=pd.concat([matched_treated,matched_control],axis=0)

In [ ]:
matched_final.sample(3)

In [ ]:
print("Rows before matching:",lalonde.shape[0])
print("Rows after matching:",matched_final.shape[0])

## Check density after matching

In [ ]:
plot_dens(matched_final,matched_final["Training"],confounders2)

In [ ]:
plot_dens(matched_final,matched_final["Training"],['ps'])

## Standardized means pre and post mathing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def love_plot(before_matching, after_matching, feature_names):
    """
    Create a vertical love plot to visualize standardized mean differences before and after matching.

    Parameters:
    - before_matching: List or array of standardized mean differences before matching.
    - after_matching: List or array of standardized mean differences after matching.
    - feature_names: List of feature names corresponding to the differences.

    Returns:
    - None (displays the plot).
    """
    num_features = len(feature_names)

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(4, 6))

    # Plot standardized mean differences before and after matching as lines
    ax.plot(before_matching, np.arange(num_features), marker='o', label='Before Matching', linestyle='-', color='red',alpha=0.6)
    ax.plot(after_matching, np.arange(num_features), marker='o', label='After Matching', linestyle='-', color='blue',alpha=0.6)

    # Annotate points with feature names
    for i, txt in enumerate(feature_names):
        ax.annotate(txt, (after_matching[i], i), textcoords="offset points", xytext=(0,10), ha='left',size=10,fontweight='bold')

    # Add labels and title
    ax.set_yticks(np.arange(num_features))
    ax.set_yticklabels([])
    ax.set_ylabel('Features')
    ax.set_xlabel('Standardized Mean Difference')
    ax.set_title('Standardized Mean Differences',fontweight='bold',fontsize=14)
    ax.axvline(x=0, color='green', linestyle= 'dashed', label='Perfect balance',alpha=1)
    ax.legend()


    # Show the plot
    plt.tight_layout()
    plt.show()

# Example usage:
# Assuming you have lists `before_matching_diff` and `after_matching_diff` with standardized mean differences,
# and a list `feature_names` with the corresponding feature names.
# vertical_love_plot(before_matching_diff, after_matching_diff, feature_names)


In [ ]:
diffs_pre=[]
for c in confounders2:
  diffs_pre.append({check_balance(lalonde,c,"training"):c})

diffs_post=[]
for c in confounders2:
  diffs_post.append({check_balance(matched_final,c,"Training"):c})

In [ ]:
names=[list(item.values())[0] for item in diffs_pre]
before_m=[list(item.keys())[0] for item in diffs_pre]
after_m=[list(item.keys())[0] for item in diffs_post]

In [ ]:
love_plot(before_m, after_m,names)

## Calculating the KS statistics for each covariate

In [ ]:
from scipy.stats import ks_2samp
import numpy as np

def compare_distributions(sample1, sample2, alpha=0.05):
    """
    Compare two distributions using the Kolmogorov-Smirnov (KS) statistic.

    Parameters:
    - sample1: First sample.
    - sample2: Second sample.
    - alpha: Significance level for hypothesis testing. Default is 0.05.

    Returns:
    - result: A string indicating the result of the KS test.
    """

    # Perform the KS test
    ks_statistic, p_value = ks_2samp(sample1, sample2)

    # Interpretation
    if p_value > alpha:
        result = 'Fail to reject the null hypothesis. The distributions are similar.'
    else:
        result = 'Reject the null hypothesis. The distributions are significantly different.'

    return result,ks_statistic


In [ ]:
vars_to_ks=["age","education","re74","re75"]
for v in vars_to_ks:
  control=matched_final[matched_final["Training"]==0][v]
  treat=matched_final[matched_final["Training"]==1][v]
  print(v)
  display(compare_distributions(control,treat, alpha=0.05))


## Applaying A/B test

In [ ]:
import numpy as np
from scipy.stats import ttest_ind, shapiro, mannwhitneyu, norm

def perform_ab_test(control_group, treatment_group, alpha=0.05):
    """
    Perform A/B test using the t-test method after checking for normality.

    Parameters:
    - control_group: 1D array or list, the data for the control group.
    - treatment_group: 1D array or list, the data for the treatment group.
    - alpha: float, significance level (default is 0.05).

    Returns:
    - result: string, indicating the result of the A/B test.
    - p_value: float, p-value from the t-test.
    """
    # Check normality using Shapiro-Wilk test
    _, control_p_value = shapiro(control_group)
    _, treatment_p_value = shapiro(treatment_group)

    # Perform t-test only if both groups are normally distributed
    if control_p_value > alpha and treatment_p_value > alpha:
        t_stat, p_value = ttest_ind(control_group, treatment_group)

        # Check significance
        if p_value < alpha:
            result = "Statistically significant difference (Reject H0)"
        else:
            result = "No statistically significant difference (Fail to reject H0)"
    else:
      # Perform Mann-Whitney U test
      U_stat, p_value = mannwhitneyu(control_group, treatment_group, alternative='two-sided')
      # Check significance
      if p_value < alpha:

        result = "Statistically significant difference (Reject H0)"
      else:
        result = "No statistically significant difference (Fail to reject H0)"

    return result, p_value

In [ ]:
re78_t=matched_final[matched_final["Training"]==0]["re78"]
re78_c=matched_final[matched_final["Training"]==1]["re78"]

In [ ]:
(sum(re78_t)/len(re78_t))-(sum(re78_c)/len(re78_c))

In [ ]:
perform_ab_test(re78_c,re78_t)

In [ ]:
re78_c_bf=lalonde[lalonde["training"]==0]["re78"]
re78_t_bf=lalonde[lalonde["training"]==1]["re78"]
perform_ab_test(re78_c_bf,re78_t_bf)

In [ ]:
from scipy.stats import mannwhitneyu, norm

def perform_mannwhitneyu_test_large_sample(control_group, treatment_group, alpha=0.05):
    """
    Perform Mann-Whitney U test with Z approximation for large sample sizes.

    Parameters:
    - control_group: 1D array or list, the data for the control group.
    - treatment_group: 1D array or list, the data for the treatment group.
    - alpha: float, significance level (default is 0.05).

    Returns:
    - result: string, indicating the result of the Mann-Whitney U test.
    - p_value: float, p-value from the test.
    """
    # Perform Mann-Whitney U test
    U_stat, p_value = mannwhitneyu(control_group, treatment_group, alternative='two-sided')

    # For large samples, use Z approximation
    n1 = len(control_group)
    n2 = len(treatment_group)
    Z_approx = (U_stat - (n1 * n2 / 2)) / np.sqrt((n1 * n2 * (n1 + n2 + 1)) / 12)

    # Calculate p-value using the Z approximation
    p_value_z_approx = 2 * (1 - norm.cdf(np.abs(Z_approx)))

    # Check significance
    if p_value_z_approx < alpha:
        result = "Statistically significant difference (Reject H0)"
    else:
        result = "No statistically significant difference (Fail to reject H0)"

    return result, p_value_z_approx



In [ ]:
(sum(re78_c_bf)/len(re78_c_bf))-(sum(re78_t_bf)/len(re78_t_bf))

In [ ]:
perform_mannwhitneyu_test_large_sample(re78_c_bf,re78_t_bf)